In [ ]:
import cv2
import numpy as np
from scipy.spatial.transform import Rotation as R
from lovely_utils.camera.calibrator.pinhole_calibrator import PinholeCalibrator

generate_pixel_to_world_map = PinholeCalibrator.generate_map_transform_pixel_points_to_world_points 
save_xyz_map_binary = PinholeCalibrator.save_map
load_xyz_map_binary = PinholeCalibrator.load_map
analyze_pixel_world_coords = PinholeCalibrator.validate_map
transfrom_RTcw_to_RTwc = PinholeCalibrator.invert_pose_transform

# def generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0):
#     H, W = image_size
#     xyz_map = np.zeros((H, W, 3), dtype=np.float32)

#     # 像素坐标网格
#     u, v = np.meshgrid(np.arange(W), np.arange(H))  # shape: (H, W)
#     pixel_coords = np.stack([u, v], axis=-1).astype(np.float32)  # (H, W, 2)
#     pixel_coords_flat = pixel_coords.reshape(-1, 1, 2)

#     # 去畸变得到归一化相机坐标
#     undistorted_pts = cv2.undistortPoints(pixel_coords_flat, K, D)  # (N,1,2)
#     undistorted_pts = undistorted_pts.reshape(-1, 2)

#     # 单位射线
#     rays_cam = np.concatenate([undistorted_pts, np.ones((undistorted_pts.shape[0], 1))], axis=1)

#     # 相机坐标系 → 世界坐标系的射线方向
#     rays_world = (R_wc @ rays_cam.T).T  # (N,3)

#     # ✅ 正确计算相机中心
#     cam_center = t_wc.flatten()  # shape (3,)

#     # 与 z=plane_z 的平面求交
#     t_vals = (plane_z - cam_center[2]) / rays_world[:, 2]
#     pts_world = cam_center + rays_world * t_vals[:, np.newaxis]

#     xyz_map = pts_world.reshape(H, W, 3).astype(np.float32)
#     return xyz_map


# def save_xyz_map_binary(xyz_map: np.ndarray, file_path: str):
#     # 确保是 float32 类型，等价于 CV_32FC3
#     if xyz_map.dtype != np.float32:
#         xyz_map = xyz_map.astype(np.float32)

#     # 写入二进制文件
#     with open(file_path, "wb") as f:
#         f.write(xyz_map.tobytes())


# def load_xyz_map_binary(file_path: str, height: int = 480, width: int = 640) -> np.ndarray:
#     """
#     加载用 save_xyz_map_binary 保存的二进制XYZ地图

#     参数:
#         file_path: 二进制文件路径
#         height: 原始数组的高度（行数）
#         width: 原始数组的宽度（列数）

#     返回:
#         xyz_map: 形状为 (height, width, 3) 的 np.float32 数组
#     """
#     # 读取二进制数据
#     with open(file_path, "rb") as f:
#         data = f.read()

#     # 转换为np.float32数组，并reshape为 (height, width, 3)
#     xyz_map = np.frombuffer(data, dtype=np.float32).reshape(height, width, 3)

#     return xyz_map


# def compute_world_projection_errors(img_points, obj_points, xyz_map):
#     """
#     计算像素反投影到世界坐标后，与原始3D目标点之间的欧式距离误差。

#     参数:
#     - img_points: (N, 2) 图像中的像素点，二维数组。
#     - obj_points: (N, 3) 对应的世界坐标点，三维数组。
#     - xyz_map: (H, W, 3) 每个像素对应的世界坐标反投影结果。

#     返回:
#     - dist_errors: 每个点的欧式距离误差（单位：米）
#     - mean_error: 平均误差
#     """
#     reprojected_world_points = []

#     for px in img_points:
#         u, v = int(round(px[0])), int(round(px[1]))
#         if 0 <= v < xyz_map.shape[0] and 0 <= u < xyz_map.shape[1]:
#             pt_world = xyz_map[v, u]  # 注意顺序是[y, x]
#             reprojected_world_points.append(pt_world)
#         else:
#             reprojected_world_points.append([np.nan, np.nan, np.nan])  # 越界处理

#     reprojected_world_points = np.array(reprojected_world_points)
#     dist_errors = np.linalg.norm(obj_points - reprojected_world_points, axis=1)
#     mean_error = np.nanmean(dist_errors)

#     return dist_errors, mean_error


def calib_pinhole_extrinsic(K: np.ndarray, D: np.ndarray, img_points: np.ndarray, obj_points: np.ndarray):
    # 第一步：将畸变像素点转换为去畸变的归一化点
    # 返回去畸变后的像素点
    undistorted_pts = cv2.undistortPoints(img_points.reshape(-1, 1, 2), K, D, P=K)
    # # reshape成 Nx2
    undistorted_pts = undistorted_pts.reshape(-1, 2)
    retval, rvec, tvec = cv2.solvePnP(obj_points, undistorted_pts, K, distCoeffs=None, flags=cv2.SOLVEPNP_ITERATIVE)
    # 计算重投影误差
    img_points_proj, _ = cv2.projectPoints(obj_points, rvec, tvec, K, distCoeffs=None)
    img_points_proj = img_points_proj.reshape(-1, 2)
    errors = np.linalg.norm(undistorted_pts - img_points_proj, axis=1)
    mean_error = np.mean(errors)
    return mean_error, rvec, tvec


# def transfrom_RTcw_to_RTwc(R_cw: np.ndarray, t_cw: np.ndarray) -> tuple:
#     """
#     将相机坐标系到世界坐标系的变换矩阵转换为世界坐标系到相机坐标系的变换矩阵。

#     参数:
#     - R_tc: 相机到世界的旋转矩阵 (3, 3)
#     - t_tc: 相机到世界的平移向量 (3,)

#     返回:
#     - R_wc: 世界到相机的旋转矩阵 (3, 3)
#     - t_wc: 世界到相机的平移向量 (3,)
#     """
#     R_wc = R_cw.T
#     t_wc = -R_cw.T @ t_cw
#     return R_wc, t_wc


# def analyze_pixel_world_coords(
#     img_points: np.ndarray,
#     obj_points: np.ndarray,
#     xyz_map: np.ndarray,
# ):
#     """
#     分析像素坐标对应的世界坐标，并计算投影误差

#     :param img_points: 图像特征点坐标列表/数组 (Nx2)
#     :param obj_points: 世界坐标系特征点坐标列表/数组 (Nx3)
#     :param xyz_map: 像素到世界坐标的映射矩阵 (HxWx3)
#     :param save_bin_path: 保存xyz_map二进制文件的路径（为None则不保存）
#     :return: 距离误差列表和平均误差
#     """
#     # 确保输入是numpy数组格式
#     img_points = np.array(img_points) if not isinstance(img_points, np.ndarray) else img_points
#     obj_points = np.array(obj_points) if not isinstance(obj_points, np.ndarray) else obj_points

#     # 检查输入维度是否匹配
#     if len(img_points) != len(obj_points):
#         raise ValueError(f"图像点与世界点数量不匹配: {len(img_points)} vs {len(obj_points)}")

#     # 打印像素坐标对应的世界坐标
#     print("像素坐标对应的世界坐标:")
#     for px, obj_p in zip(img_points, obj_points):
#         # 转换为整数像素坐标（四舍五入）
#         u, v = int(round(px[0])), int(round(px[1]))

#         # 获取映射的世界坐标和实际世界坐标
#         x, y, z = xyz_map[v, u]
#         x_obj, y_obj, z_obj = obj_p

#         # 格式化输出（保留3位小数）
#         print(
#             f"Pixel ({v}, {u}) → 映射坐标: ({x:.3f}, {y:.3f}, {z:.3f}) | 实际坐标: ({x_obj:.3f}, {y_obj:.3f}, {z_obj:.3f})"
#         )

#     # 计算投影误差
#     dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

#     # 打印误差结果
#     print("\n📏 3D 世界坐标误差 (单位：米):")
#     for i, err in enumerate(dist_errors):
#         print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
#     print(f"  平均距离误差 = {mean_error:.4f} m")

#     return


PLANE_Z = -0.07

In [3]:
# front
K = np.array([[571.0, 0.0, 329.86688232], [0.0, 571.0, 239.08282471], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array(
    [
        [2.50, 0.288, -0.07],
        [2.50, -0.612, -0.07],
        [1.60, -0.612, -0.07],
        [1.60, 0.288, -0.07],
    ],
    dtype=np.float32,
)

# 图像中的像素坐标（c）
img_points = np.array(
    [
        [270.0, 290.0],
        [507.0, 295.0],
        [607.0, 465.0],
        [224.0, 456.0],
    ],
    dtype=np.float32,
)

rms, rvec, tvec = calib_pinhole_extrinsic(K, D, img_points, obj_points)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)

R_cw, _ = cv2.Rodrigues(rvec)
t_cw = tvec
R_wc, t_wc = transfrom_RTcw_to_RTwc(R_cw, tvec)

image_size = (480, 640)  # H, W
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=PLANE_Z)

analyze_pixel_world_coords(
    img_points=img_points,
    obj_points=obj_points,
    xyz_map=xyz_map,
)



像素坐标对应的世界坐标:
Pixel (290, 270) → 映射坐标: (2.502, 0.291, -0.070) | 实际坐标: (2.500, 0.288, -0.070)
Pixel (295, 507) → 映射坐标: (2.497, -0.614, -0.070) | 实际坐标: (2.500, -0.612, -0.070)
Pixel (465, 607) → 映射坐标: (1.601, -0.612, -0.070) | 实际坐标: (1.600, -0.612, -0.070)
Pixel (456, 224) → 映射坐标: (1.599, 0.287, -0.070) | 实际坐标: (1.600, 0.288, -0.070)

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.0039 m
  点 2: 距离误差 = 0.0041 m
  点 3: 距离误差 = 0.0009 m
  点 4: 距离误差 = 0.0009 m
  平均距离误差 = 0.0024 m


In [4]:
# front
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 329.86688232], [0.0, 571.0, 239.08282471], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = 69.36901996548279, 179.9543793453007, 91.40716490747336
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[0.4752837 + 0.06], [-0.0002232025], [0.9815271]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=PLANE_Z)

save_xyz_map_binary(xyz_map, "front_map_808.bin")

Map saved to front_map_808.bin


In [ ]:
# back
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array(
    [
        [566.72375488, 0.0, 333.63687134],
        [0.0, 566.10162354, 230.97738647],
        [0.0, 0.0, 1.0],
    ]
)
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = -116.1267943062868, 1.878901157409764, 90.93493245844785
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[-0.1393937 + 0.06], [0.009496391], [0.878864]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=PLANE_Z)

save_xyz_map_binary(xyz_map, "back_map_808.bin")
# 世界坐标系中的棋盘格角点（Z=0）

In [ ]:
# left
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 338.2255249], [0.0, 571.0, 234.12065125], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = 30.26064768374291, 166.3358837252657, 110.9936407676445
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[0.4364802 + 0.06], [0.1249509], [1.014549]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0)

save_xyz_map_binary(xyz_map, "left_map_808.bin")
# 世界坐标系中的棋盘格角点（Z=0）

In [ ]:
# right
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array(
    [
        [582.09985352, 0.0, 334.96206665],
        [0.0, 581.71112061, 238.98794556],
        [0.0, 0.0, 1.0],
    ]
)
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = 29.32168108797375, -166.1284918613074, 69.14326855303553
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[0.4476331 + 0.06], [-0.08687994], [0.9935386]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=PLANE_Z)

save_xyz_map_binary(xyz_map, "right_map_808.bin")
# 世界坐标系中的棋盘格角点（Z=0）

In [ ]:
def compare_xyz_maps(
    file_path1: str,
    file_path2: str,
    height: int,
    width: int,
    rtol: float = 1e-5,
    atol: float = 1e-8,
) -> bool:
    """
    比较两个XYZ地图文件是否一致

    参数:
        file_path1, file_path2: 两个二进制文件路径
        height, width: 点云高度和宽度（需已知）
        rtol, atol: 相对和绝对容差（用于浮点数比较）

    返回:
        is_equal: 是否一致
        diff_stats: 差异统计信息（如果不一致）
    """
    # 加载两个文件
    map1 = load_xyz_map_binary(file_path1, height, width)
    map2 = load_xyz_map_binary(file_path2, height, width)

    # 检查形状和数据类型
    if map1.shape != map2.shape or map1.dtype != map2.dtype:
        return False, {
            "reason": "Shape or dtype mismatch",
            "shape1": map1.shape,
            "shape2": map2.shape,
            "dtype1": map1.dtype,
            "dtype2": map2.dtype,
        }

    # 逐元素比较（使用numpy.allclose处理浮点数精度问题）
    is_equal = np.allclose(map1, map2, rtol=rtol, atol=atol)

    if not is_equal:
        # 计算差异统计信息
        diff = np.abs(map1 - map2)
        max_diff = np.max(diff)
        mean_diff = np.mean(diff)
        diff_percentage = np.count_nonzero(diff > atol) / diff.size * 100

        return False, {
            "reason": "Element-wise mismatch",
            "max_difference": max_diff,
            "mean_difference": mean_diff,
            "difference_percentage": diff_percentage,  # 差异元素的百分比
            "example_diff": diff[np.unravel_index(np.argmax(diff), diff.shape)],
        }

    return True, {}

In [ ]:
# 假设两个文件的形状都是 (480, 640, 3)
height, width = 480, 640
file1 = "/home/ubuntu/Desktop/project/sensor-calibration/notebooks/front_map_test.bin"
file2 = "/home/ubuntu/Desktop/project/sensor-calibration/tmp/front_map_wc.bin"

# 比较两个文件
is_equal, diff_info = compare_xyz_maps(file1, file2, height, width)

if is_equal:
    print("两个文件完全一致")
else:
    print("两个文件存在差异:")
    for key, value in diff_info.items():
        print(f"  {key}: {value}")

两个文件存在差异:
  reason: Element-wise mismatch
  max_difference: 11632650.0
  mean_difference: 17.696548461914062
  difference_percentage: 66.56022135416667
  example_diff: 11632650.0


In [ ]:
file1 = "/home/ubuntu/Desktop/project/sensor-calibration/notebooks/front_map_test.bin"
file2 = "/home/ubuntu/Desktop/project/sensor-calibration/tmp/front_map_wc.bin"
map_1 = load_xyz_map_binary(file1, height=480, width=640)
map_2 = load_xyz_map_binary(file2, height=480, width=640)
print(map_1.shape)
print(map_2.shape)

img_points = np.array(
    [
        [262, 291],
        [499, 296.0],
        [595, 464.0],
        [212, 457.0],
    ],
    dtype=np.float32,
)
obj_points = np.array(
    [
        [2.50, 0.31, -0.07],
        [2.50, -0.59, -0.07],
        [1.60, -0.59, -0.07],
        [1.60, 0.31, -0.07],
    ],
    dtype=np.float32,
)

for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("map_1:", map_1[v, u])
    print("map_2:", map_2[v, u])
# 比较两个文件
is_equal, diff_info = compare_xyz_maps(file1, file2, height, width)

if is_equal:
    print("两个文件完全一致")
else:
    print("两个文件存在差异:")
    for key, value in diff_info.items():
        print(f"  {key}: {value}")

(480, 640, 3)
(480, 640, 3)
map_1: [ 2.6306937  0.2967763 -0.07     ]
map_2: [ 2.63069     0.29677552 -0.07      ]
map_1: [ 2.552405  -0.6860397 -0.07     ]
map_2: [ 2.5524015 -0.6860388 -0.07     ]
map_1: [ 1.5905168 -0.6502192 -0.07     ]
map_2: [ 1.5905149  -0.65021867 -0.07      ]
map_1: [ 1.6383387   0.32942784 -0.07      ]
map_2: [ 1.6383369   0.32942706 -0.07      ]
两个文件存在差异:
  reason: Element-wise mismatch
  max_difference: 11632650.0
  mean_difference: 17.696548461914062
  difference_percentage: 66.56022135416667
  example_diff: 11632650.0


In [ ]:
# 比较深度相机标定结果的差异
# front
# Step 1: 正常标定流程得到外参
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 329.86688232], [0.0, 571.0, 239.08282471], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array(
    [
        [2.50, 0.31, -0.07],
        [2.50, -0.59, -0.07],
        [1.60, -0.59, -0.07],
        [1.60, 0.31, -0.07],
    ],
    dtype=np.float32,
)

# 图像中的像素坐标（c）
img_points = np.array(
    [
        [262, 291],
        [499, 296.0],
        [595, 464.0],
        [212, 457.0],
    ],
    dtype=np.float32,
)

rms, rvec, tvec = calib_pinhole_extrinsic(K, D, img_points, obj_points)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)

R_cw, _ = cv2.Rodrigues(rvec)
t_cw = tvec
R_wc, t_wc = transfrom_RTcw_to_RTwc(R_cw, tvec)

image_size = (480, 640)  # H, W
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=PLANE_Z)

analyze_pixel_world_coords(
    img_points=img_points,
    obj_points=obj_points,
    xyz_map=xyz_map,
)
print("前视RGB相机标定结果:")
print("R_wc:\n", R_wc, "\n t_wc:\n", t_wc)
print("R_cw:\n", R_cw, "\n t_cw:\n", t_cw)

像素坐标对应的世界坐标:
Pixel (291, 262) → 映射坐标: (2.506, 0.313, -0.070) | 实际坐标: (2.500, 0.310, -0.070)
Pixel (296, 499) → 映射坐标: (2.493, -0.592, -0.070) | 实际坐标: (2.500, -0.590, -0.070)
Pixel (464, 595) → 映射坐标: (1.602, -0.590, -0.070) | 实际坐标: (1.600, -0.590, -0.070)
Pixel (457, 212) → 映射坐标: (1.598, 0.309, -0.070) | 实际坐标: (1.600, 0.310, -0.070)

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.0068 m
  点 2: 距离误差 = 0.0071 m
  点 3: 距离误差 = 0.0018 m
  点 4: 距离误差 = 0.0017 m
  平均距离误差 = 0.0044 m
前视RGB相机标定结果:
R_wc:
 [[ 0.02356982 -0.34748667  0.93738865]
 [-0.99970862 -0.0130787   0.02028857]
 [ 0.00520982 -0.93759371 -0.34769368]] 
 t_wc:
 [[0.51795576]
 [0.00983438]
 [0.88445068]]
R_cw:
 [[ 0.02356982 -0.99970862  0.00520982]
 [-0.34748667 -0.0130787  -0.93759371]
 [ 0.93738865  0.02028857 -0.34769368]] 
 t_cw:
 [[-0.00698444]
 [ 1.00936674]
 [-0.17820746]]


In [21]:
# Step 2: 使用深度相机标定结果获取外参
image_size = (480, 640)  # H, W
roll, pitch, yaw = 69.36901996548279, 179.9543793453007, 91.40716490747336
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[0.4752837+0.02], [-0.0002232025], [0.9815271 - 0.07]])
R_cw, t_cw = transfrom_RTcw_to_RTwc(R_wc, t_wc)
# t_cw[0 ] = t_cw[0] + 0.05  # 调整平移向量
R_wc, t_wc = transfrom_RTcw_to_RTwc(R_cw, t_cw)
image_size = (480, 640)  # H, W
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=PLANE_Z)
print("前视深度相机标定结果:")
print("R_wc:\n", R_wc, "\n t_wc:\n", t_wc)
print("R_cw:\n", R_cw, "\n t_cw:\n", t_cw)

# save_xyz_map_binary(xyz_map, "front_map_test.bin")
analyze_pixel_world_coords(
    img_points=img_points,
    obj_points=obj_points,
    xyz_map=xyz_map,
)

前视深度相机标定结果:
R_wc:
 [[ 2.45571841e-02 -3.52259768e-01  9.35580034e-01]
 [-9.99698110e-01 -7.90772786e-03  2.32627839e-02]
 [-7.96230547e-04 -9.35868860e-01 -3.52347616e-01]] 
 t_wc:
 [[ 4.952837e-01]
 [-2.232025e-04]
 [ 9.115271e-01]]
R_cw:
 [[ 2.45571841e-02 -9.99698110e-01 -7.96230547e-04]
 [-3.52259768e-01 -7.90772786e-03 -9.35868860e-01]
 [ 9.35580034e-01  2.32627839e-02 -3.52347616e-01]] 
 t_cw:
 [[-0.01166012]
 [ 1.02753658]
 [-0.14219795]]
像素坐标对应的世界坐标:
Pixel (290, 270) → 映射坐标: (2.526, 0.287, -0.070) | 实际坐标: (2.500, 0.288, -0.070)
Pixel (295, 507) → 映射坐标: (2.503, -0.636, -0.070) | 实际坐标: (2.500, -0.612, -0.070)
Pixel (465, 607) → 映射坐标: (1.592, -0.632, -0.070) | 实际坐标: (1.600, -0.612, -0.070)
Pixel (456, 224) → 映射坐标: (1.601, 0.285, -0.070) | 实际坐标: (1.600, 0.288, -0.070)

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.0263 m
  点 2: 距离误差 = 0.0239 m
  点 3: 距离误差 = 0.0210 m
  点 4: 距离误差 = 0.0032 m
  平均距离误差 = 0.0186 m
